<a href="https://colab.research.google.com/github/tanongsakintean/google_colab/blob/main/pyarrowTanongsakIntean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.compute as pc

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #ระวังเครื่องหมาย ‘ กับ ' ต่างกันนะคะ

In [ ]:
import pandas as pd
path = '/content/drive/MyDrive/Colab Notebooks/dataset/people-2000000.csv'
dfp = pd.read_csv(path) ## Dataset is now stored in a Pandas Dataframe
dfp

1.  สร้าง partition  file  เลือก   Index ชื่อ นามสกุล เพศ วันเดือนปีเกิด และ งานที่ทำ  โดยกรองในส่วนของ Job Title เป้นหลัก


In [ ]:
# Save the table to a Parquet file

dfp.to_parquet('sample.parquet', index=False) # save to parquet format
# Load the table from the Parquet file
table2 = pq.read_table('sample.parquet')
table2

In [ ]:
# write to multiple partitioned files
pq.write_to_dataset(table2.select(["Index", "First Name","Last Name","Sex","Job Title"]), root_path='my_parq', partition_cols=['Job Title'])

In [ ]:
table3 = pq.read_table('my_parq')
table3.to_pandas()

2.ให้ตรวจสอบตำแหน่งงานที่มีในรายการว่ามีทั้งหมดกี่ตำแหน่งงาน แต่ละตำแหน่งงานมีจำนวนกี่คน

In [ ]:
grouped_table = table3.group_by("Job Title").aggregate(
  [("Index",
  "count",  #count #max #min

  pc.CountOptions(mode="only_valid"))] # เอาเฉพาะที่มีข้อมูลนะ
)
# group by country โดย หา sum ของ Number of employees.
grouped_table.to_pandas()



3.กรองข้อมูลโดยแสดงเฉพาะข้อมูลของคนที่เป็นเพศชายและมีตำแหน่งเป็น "Writer" เท่านั้น

In [ ]:
expr1 = pc.field("Sex") == "Male"

In [ ]:
expr2 = pc.field("Job Title") == "Writer"

In [ ]:
filtered_df = table3.filter(expr1 & expr2)

In [ ]:
filtered_df.to_pandas()

In [ ]:
table3